In [ ]:
import requests
import csv
import re
from collections import defaultdict
from google.colab import files

def descargar_archivo_estacion(estaciones):
    # Procesar las entradas para soportar rangos con guiones
    estaciones_procesadas = []
    for estacion in estaciones:
        if isinstance(estacion, str) and "-" in estacion:
            inicio, fin = map(int, estacion.split("-"))
            estaciones_procesadas.extend(range(inicio, fin + 1))
        else:
            estaciones_procesadas.append(int(estacion))

    for estacion in estaciones_procesadas:
        # URL del archivo a descargar
        url = f"https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Mensuales/mich/mes{estacion:05d}.txt"

        try:
            # Realizar la solicitud GET a la URL
            response = requests.get(url)
            response.raise_for_status()  # Verificar si la solicitud fue exitosa

            # Procesar el contenido del archivo descargado
            lines = response.text.splitlines()

            # Obtener el número de estación, nombre de la estación, fecha de emisión, latitud, longitud y altitud de las primeras líneas
            numero_estacion = "desconocida"
            nombre_estacion = "desconocido"
            fecha_emision = "sin_fecha"
            latitud = ""
            longitud = ""
            altitud = ""
            for line in lines[:30]:  # Revisar más de 23 líneas para asegurarse de encontrar toda la información relevante
                if "ESTACIÓN" in line:
                    numero_estacion = line.split(":")[-1].strip()
                elif "NOMBRE" in line:
                    nombre_estacion = line.split(":")[-1].strip().replace(" ", "_")
                elif "EMISIÓN" in line:
                    fecha_emision = line.split(":")[-1].strip().replace("/", "-")
                elif "LATITUD" in line:
                    latitud = line.split(":")[-1].strip().replace(" °", "")
                elif "LONGITUD" in line:
                    longitud = line.split(":")[-1].strip().replace(" °", "")
                elif "ALTITUD" in line:
                    altitud = line.split(":")[-1].strip().replace(" msnm", "")

            file_name = f"{numero_estacion}_{nombre_estacion}_{fecha_emision}.csv"

            # Encontrar todas las secciones con datos (las líneas antes de "AÑO" son los títulos de cada sección)
            conjuntos = []
            abreviaciones = []
            headers = []
            idx = 0
            min_year = float('inf')
            max_year = float('-inf')

            while idx < len(lines):
                line = lines[idx].strip()
                # Identificar el título de una sección si la línea siguiente contiene "AÑO"
                if line and idx + 1 < len(lines) and "AÑO" in lines[idx + 1]:
                    conjunto_name = line
                    conjuntos.append(conjunto_name)
                    abreviaciones.append(re.sub(r'[^A-Za-z0-9]+', '', conjunto_name).lower())
                    headers = [
                        "AÑO", "ENE", "FEB", "MAR", "ABR", "MAY", "JUN", "JUL", "AGO", "SEP", "OCT", "NOV", "DIC", "ACUM", "PROM", "MESES"
                    ]
                    idx += 2  # Saltar el encabezado de columnas (AÑO, ENE, FEB, ...)
                    # Obtener los años mínimo y máximo
                    temp_idx = idx
                    while temp_idx < len(lines):
                        temp_line = lines[temp_idx].strip()
                        if re.match(r'^\d{4}', temp_line):
                            year = int(temp_line.split()[0])
                            min_year = min(min_year, year)
                            max_year = max(max_year, year)
                        elif temp_line == "" or any(conj in temp_line for conj in conjuntos):
                            break  # Salir del bucle si hay una línea vacía o si empieza otro conjunto
                        temp_idx += 1
                else:
                    idx += 1


            # Inicializar un diccionario para almacenar todos los datos
            data_dict = defaultdict(lambda: {header: "null" for header in headers})

            # Recorrer todos los conjuntos de datos y extraer la información
            idx = 0
            current_conjunto = None
            while idx < len(lines):
                line = lines[idx].strip()
                if line in conjuntos:
                    current_conjunto = next(
                        abreviacion for conjunto, abreviacion in zip(conjuntos, abreviaciones)
                        if conjunto == line
                    )
                    idx += 2  # Saltar el encabezado de columnas (AÑO, ENE, FEB, ...)
                elif current_conjunto and re.match(r'^\d{4}', line):
                    values = re.split(r'\s+', line)
                    if len(values) >= 1 and values[0].isdigit():  # Año y demás valores
                        year = int(values[0])
                        if min_year <= year <= max_year:
                            if year not in data_dict:
                                data_dict[year] = {header: "null" for header in headers}
                            for idx_value, header in enumerate(headers):
                                if idx_value < len(values):
                                    value = values[idx_value]
                                    field_name = f"{current_conjunto}_{header}"
                                    data_dict[year][field_name] = value if value else "null"
                            data_dict[year]["AÑO"] = year
                idx += 1

            # Crear los encabezados
            final_headers = ["AÑO"] + [f"{abbr}_{month}" for abbr in abreviaciones for month in [
                "ENE", "FEB", "MAR", "ABR", "MAY", "JUN", "JUL", "AGO", "SEP", "OCT", "NOV", "DIC", "ACUM", "PROM", "MESES"
            ]] + ["LATITUD", "LONGITUD", "ALTITUD"]

            # Escribir el CSV
            with open(file_name, "w", newline="", encoding="utf-8-sig") as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(final_headers)
                for year in range(min_year, max_year + 1):
                    row = [year] + [data_dict[year].get(header, "null") for header in final_headers[1:-3]] + [latitud, longitud, altitud]
                    writer.writerow(row)

            # Descargar el archivo en la PC del usuario desde Google Colab
            files.download(file_name)

            print(f"Archivo descargado y convertido a CSV con éxito: {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error al acceder a la URL {url}: {e}")

# Ejemplo de uso
descargar_archivo_estacion(["16048-16050"])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Archivo descargado y convertido a CSV con éxito: 16048_ZAMORA_(DGE)_25-10-2024.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Archivo descargado y convertido a CSV con éxito: 16049_ETUCUARO_25-10-2024.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Archivo descargado y convertido a CSV con éxito: 16050_HUANIQUEO_25-10-2024.csv
